Exploration/Visualization of the data

In [ ]:
import pandas as pd
import numpy
import matplotlib as plt

Plot household income to check for extreme outliers

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(final_df["household_income"], bins=100, edgecolor='black')
plt.title("Distribution of Household Income")
plt.xlabel("Household Income")
plt.ylabel("Number of Households")
plt.grid(axis='y', alpha=0.3)
plt.show()